# Podstawy podejmowania decyzji projekt
283018

## Opis

1. Plansza (macierz) nxn zapełniona symbolami.
2. Każda komórka macierzy zawiera jeden symbol w formacie heksadecymalnym (nie ma to zasadniczego znaczenia i można je zamienić na liczby całkowite lub symbole ascii) z pewnej ograniczonej puli.
3. Symbole w różnych komórkach macierzy mogą się powtarzać.
4. Jest lista "demonów" - sekwencji różnej długości złożonych z symboli z tej samej puli (przy czym jedna sekwencja może zawierać inną sekwencję w części albo w pełni).
5. Jest ograniczony "buffor" - maksymalna ilość symboli które można aktywować/wybrać na macierzy.
6. Wybierać symbole z planszy można według określonych zasad:<br>
	a) w pierwszym kroku można wybrać dowolny symbol z pierwszego *wiersza*, na przykład (0, 4)<br>
	b) w drugim kroku symbol z tej samej *kolumny* co poprzednio wybrany symbol, np. (3, 4)<br>
	c) w trzecim znowu z tego samego *wiersza* co poprzednio wybrany, np. (5, 4)<br>
	Na ogół, jeśli krok jest nieparzysty to wybór się odbywa w odpowiednim (zgodnym z poprzednio wybranym) *wierszu*, jeśli parzysty to w *kolumnie*.
7. Można wybierać symbol z tej samej komórki macierzy tylko jednokrotnie
8. Jeśli zostały kolejno wybrane symbole odpowiadające "demonowi" to ta sekwencja jest "aktywowana".
9. Można wybierać symbole niebędące elementem żadnego "demona".
10. Macierz jest generowana w taki sposób by każdy pojedynczy "demon" był możliwy do aktywacji

# Instalacja

In [ ]:
!pip install gurobipy
!pip install icecream


In [2]:
import gurobipy as gp
import numpy as np  # noqa: F401
from icecream import ic  # noqa: F401
from pprint import pprint as pp #noqa: F401
# import matplotlib.pyplot as plt

# ------

In [53]:
# dane wejściowe
buffer_size = 8
demons = [
    [1, 3, 4, 1],
    [3, 1, 2],
    [1, 3, 1, 2, 4],
]
d_costs = [3, 2, 5]

# TODO: numpy array?
matrix = [
    [1, 3, 3],
    [4, 2, 1],
    [3, 1, 4]
]

In [64]:
# aktualna konfiguracja (stałe)
d_amo = len(demons)
d_lengths = [len(i) for i in demons]
matrix_size = len(matrix)
if not all(len(row) == matrix_size for row in matrix):
    raise ValueError("Matrix must have same dimensions")



model = gp.Model("Breach Protocol")
model.setParam("OutputFlag", 0)



# wybór drogi
x = model.addVars(matrix_size, matrix_size, buffer_size, vtype=gp.GRB.BINARY, name='x')

# tylko jedna komórka wybierana w każdym kroku
for t in range(buffer_size):
    model.addConstr(x.sum('*', '*', t) == 1, name=f'one_cell_per_step_{t}')
#TODO: brak wyboru dostępny? (jeśli tak to może być tylko na końcu)

# komórka może być wybrana co najwyżej raz
for i in range(matrix_size):
    for j in range(matrix_size):
        model.addConstr(x.sum(i, j, '*') <= 1, name=f"cell_{i}_{j}_once")

# maksymalna ilość kroków (prawdopodobnie zbędne)
model.addConstr((x.sum('*', '*', '*') <= buffer_size), name="max_steps")



# bezpośrednio sekwencja buffera
buffer_seq = [
    gp.quicksum(
        matrix[i][j] * x[i, j, t]
        for i in range(matrix_size)
        for j in range(matrix_size)
        )
    for t in range(buffer_size)
]



# lista, dla każdego demona jest słownik pozycji w sekwencji buffera gdzie ten demon może się zaczynać : is_valid
z = []
for i in range(d_amo):
    curr_len = d_lengths[i]
    valid_p = buffer_size - curr_len + 1
    z_i = {}
    for p in range(valid_p):
        z_i[p] = model.addVar(vtype=gp.GRB.BINARY, name=f"z_{i}_{p}")
    z.append(z_i)
# [{pos: is_valid}, ...] ∀ demon (according to demon length)


for i in range(d_amo):                              # ∀ demona
    curr_len = d_lengths[i]
    for p in range(buffer_size - curr_len + 1):     # ∀ valid starting pos
        for s in range(curr_len):                   # ∀ symbolu demona
            t = p + s   # krok (pozycja w buffer)
            model.addGenConstrIndicator(
                z[i][p],                        # jeśli dla tego demona dla tej pozycji startowej is_valid
                1,                              # == 1 (True)
                buffer_seq[t] == demons[i][s],  # to w buffer (na odpowiedniej pozycji) MUSI być symbol demona
                name=f"indicator(demon_{i}_pos_{p}_symbol_{s})"
            )


# wybrane demony
y = model.addVars(d_amo, vtype=gp.GRB.BINARY, name='y')


# jeśli chociaż jedna z starting_pos valid to y[i] == 1
for i in range(d_amo):
    curr_len = d_lengths[i]
    valid_p = buffer_size - curr_len + 1

    #! sekwencja demona może występować w buffer kilkakrotnie np. demon=[1, 3], buffer = [1, 3, 1, 3]
    # y[i] == 0 jeśli *żadna* z z[i][p] nie == 1
    model.addConstr(y[i] <= gp.quicksum(z[i][p] for p in range(valid_p)), name=f"active_y_{i}_at_least_one")
    # jeśli dowolna z z[i][p] == 1, y[i] też == 1
    for p in range(valid_p):
        model.addConstr(y[i] >= z[i][p], name=f"active_y_{i}_for_{p}")




# maksymalizacja punktów (obv)
model.setObjective(gp.quicksum(d_costs[i] * y[i] for i in range(d_amo)), gp.GRB.MAXIMIZE)
model.optimize()



buffer_nums = [int(buffer_seq[t].getValue()) for t in range(buffer_size)]
pp(buffer_nums)

z_nums = []
for i in range(len(z)):
    z_nums.append({})
    for j, k in z[i].items():
        z_nums[i][j] = True if k.x > 0.5 else False
pp(z_nums)

x_steps_nums = [[None for _ in range(matrix_size)] for _ in range(matrix_size)]
for i in range(matrix_size):
    for j in range(matrix_size):
        for t in range(buffer_size):
            if x[i, j, t].x > 0.5:
                x_steps_nums[i][j] = t
pp(x_steps_nums)

x_path_cells_nums = []
for t in range(buffer_size):
    for i in range(matrix_size):
        for j in range(matrix_size):
            if x[i, j, t].x > 0.5:
                x_path_cells_nums.append((i, j))
pp(x_path_cells_nums)

x_cell_chosen_num = [[0 for _ in range(matrix_size)] for _ in range(matrix_size)]
for i in range(matrix_size):
    for j in range(matrix_size):
        for t in range(buffer_size):
            if x[i, j, t].x > 0.5:
                x_cell_chosen_num[i][j] += 1
pp(x_cell_chosen_num)

y_nums = []
for i in y.values():
    y_nums.append(bool(int(i.x)))
pp(y_nums)

y_points_total = sum(d_costs[i] * y_nums[i] for i in range(d_amo))
pp(y_points_total)



# print("\n"+"#"*50)
# opt = model.status == gp.GRB.OPTIMAL
#
# if opt:
#     sol = 0
#     print("Selected")
#     for i in range(len(y)):
#         if y[i].x > 0.5:
#             print(f"{i}  (length: {d_lengths[i]}, cost: {d_costs[i]})")
# else:
#     print("No solution found")


[1, 3, 4, 1, 3, 1, 2, 4]
[{0: True, 1: False, 2: False, 3: False, 4: False},
 {0: False, 1: False, 2: False, 3: False, 4: True, 5: False},
 {0: False, 1: False, 2: False, 3: True}]
[[0, 1, 4], [2, 6, 3], [None, 5, 7]]
[(0, 0), (0, 1), (1, 0), (1, 2), (0, 2), (2, 1), (1, 1), (2, 2)]
[[1, 1, 1], [1, 1, 1], [0, 1, 1]]
[True, True, True]
10
